In [ ]:
!git clone https://github.com/WongKinYiu/yolov7.git
%cd yolov7
!pip install -r requirements.txt

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import numpy
import pandas as pd

In [3]:
!unzip ./drive/MyDrive/indabaX/train.zip

Archive:  ./drive/MyDrive/indabaX/train.zip
   creating: train/
  inflating: train/ID_000343.png     
  inflating: train/ID_000410.png     
  inflating: train/ID_000242.png     
  inflating: train/ID_000316.png     
  inflating: train/ID_000255.png     
  inflating: train/ID_000220.png     
  inflating: train/ID_000424.png     
  inflating: train/ID_000384.png     
  inflating: train/ID_000090.png     
  inflating: train/ID_000365.png     
  inflating: train/ID_000577.png     
  inflating: train/ID_000411.png     
  inflating: train/ID_000019.png     
  inflating: train/ID_000291.png     
  inflating: train/ID_000257.png     
  inflating: train/ID_000399.png     
  inflating: train/ID_000390.png     
  inflating: train/ID_000371.png     
  inflating: train/ID_000401.png     
  inflating: train/ID_000379.png     
  inflating: train/ID_000357.png     
  inflating: train/ID_000511.png     
  inflating: train/ID_000445.png     
  inflating: train/ID_000561.png     
  inflating: train/ID_00

In [4]:
path = "./drive/MyDrive/indabaX/"
images = os.listdir("./train")
train = pd.read_csv(path + "Train.csv")

In [29]:
train

,Image_ID,ToothClass,Xmin,Ymin,Xmax,Ymax
0,55,47,81,102,144,167
1,476,35,371,207,432,278
2,263,33,274,302,312,345
3,124,34,337,317,386,367
4,117,45,103,260,155,313
...,...,...,...,...,...,...
16008,1000,23,149,335,179,358
16009,358,32,271,336,312,376
16010,403,35,372,285,431,338
16011,998,26,81,187,147,262


In [5]:
%mkdir dataset

In [6]:
%mkdir dataset/images/
%mkdir dataset/labels/
%mkdir dataset/images/train/
%mkdir dataset/images/val/
%mkdir dataset/labels/train/
%mkdir dataset/labels/val/

In [7]:
train = train.sample(frac=1).reset_index(drop=True)

In [8]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(train, test_size=0.2, random_state=42)

In [9]:
import shutil

In [10]:
class_mapping = {
    11: 0, 12: 1, 13: 2, 14: 3, 15: 4, 16: 5, 17: 6, 18: 7,
    21: 8, 22: 9, 23: 10, 24: 11, 25: 12, 26: 13, 27: 14, 28: 15,
    31: 16, 32: 17, 33: 18, 34: 19, 35: 20, 36: 21, 37: 22, 38: 23,
    41: 24, 42: 25, 43: 26, 44: 27, 45: 28, 46: 29, 47: 30, 48: 31
}

def convert_to_yolo(df,class_mapping,sample):
  if sample=='train':
    labels = "./dataset/labels/train/"
    image_path = "./dataset/images/train/"
  else:
    labels = "./dataset/labels/val/"
    image_path = "./dataset/images/val/"

  for i,row in df.iterrows():
    xmin,ymin,xmax,ymax = row["Xmin"],row["Ymin"],row["Xmax"],row["Ymax"]

    Class = class_mapping[row["ToothClass"]]

    x_center = (xmin + xmax)/2 /512
    y_center = (ymin + ymax)/2 /512
    width = (xmax - xmin)/512
    height = (ymax - ymin)/512

    file = f"{labels}ID_{row['Image_ID']:06d}.txt"
    image = f"{image_path}ID_{row['Image_ID']:06d}.png"
    if not os.path.exists(image):
      shutil.copy2(f"./train/ID_{row['Image_ID']:06d}.png",image)
    with open(file, "a") as f:
      f.write(f"{Class} {x_center} {y_center} {width} {height}\n")

convert_to_yolo(train_df,class_mapping,"train")
convert_to_yolo(val_df,class_mapping,"val")

In [11]:
!touch ./dataset/train.txt
!touch ./dataset/test.txt

In [12]:
for img in images:
  file_train = "./dataset/train.txt"
  file_val = "./dataset/test.txt"
  with open(file_train, "a") as f:
    f.write(f"./dataset/images/train/{img}")
  with open(file_val, "a") as f:
    f.write(f"./dataset/images/val/{img}")


In [13]:
%mv data.yaml ./yolov7

In [14]:
%cd yolov7/

/content/yolov7


In [17]:
!python train.py --img 512 --batch 16 --epochs 100 --data ./data.yaml --cfg cfg/training/yolov7.yaml --weights yolov7.pt --cache

2024-05-21 18:13:56.263891: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-21 18:13:56.317712: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-21 18:13:56.317759: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-21 18:13:56.319724: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-21 18:13:56.328065: I tensorflow/core/platform/cpu_feature_guar

In [16]:
%pwd

'/content/yolov7'

In [ ]:
!unzip ../drive/MyDrive/indabaX/test.zip -d ../test

In [ ]:
!python detect.py --weights ./runs/train/exp2/weights/best.pt --img-size 512 --conf 0.25 --source ../test/test/ --save-txt --save-conf --name yolov7_test_inference

In [26]:
import os
import pandas as pd


class_mapping = {
    0: 11, 1: 12, 2: 13, 3: 14, 4: 15, 5: 16, 6: 17, 7: 18,
    8: 21, 9: 22, 10: 23, 11: 24, 12: 25, 13: 26, 14: 27, 15: 28,
    16: 31, 17: 32, 18: 33, 19: 34, 20: 35, 21: 36, 22: 37, 23: 38,
    24: 41, 25: 42, 26: 43, 27: 44, 28: 45, 29: 46, 30: 47, 31: 48
}

labels = os.listdir("./runs/detect/yolov7_test_inference/labels")
labels = sorted(labels)
test = pd.read_csv("../drive/MyDrive/indabaX/Test.csv")

In [35]:
labels_path = "./runs/detect/yolov7_test_inference/labels"

submission = []
for t in test["Image_ID"].to_list():
  label = f"{labels_path}/{t}.txt"
  class_detections = {}
  with open(label, "r") as f:
    for line in f.readlines():
      parts = line.strip().split()
      class_id, x_center, y_center, width, height, conf = map(float, parts)
      original_class = class_mapping[int(class_id)]

      xmin = int((x_center - width/2) * 512)
      ymin = int((y_center - height/2) * 512)

      xmax = int((x_center + width/2) * 512)
      ymax = int((y_center + height/2) * 512)

      original_class_id = f"class_" + str(original_class)
      if original_class_id not in class_detections or class_detections[original_class_id]['confidence'] < conf:
        class_detections[original_class_id] = {
          'Image_ID': t,
          'class': round(conf,1),
          'confidence': conf,
          'ymin': ymin,
          'xmin': xmin,
          'ymax': ymax,
          'xmax': xmax,
        }
  if class_detections:
        submission.extend(class_detections.values())
  else:

      submission.append({
          'Image_ID': t,
          'class': '0',
          'confidence': 0,
          'ymin': 0,
          'xmin': 0,
          'ymax': 0,
          'xmax': 0,

      })

submission_df = pd.DataFrame(submission)
submission_df = submission_df[['Image_ID', 'class', 'confidence', 'ymin', 'xmin', 'ymax', 'xmax']]
submission_df.to_csv("../submition.csv", index=False)

In [37]:
preds = pd.read_csv("../submition.csv")

In [41]:
preds.loc[preds["confidence"]<0.5]

,Image_ID,class,confidence,ymin,xmin,ymax,xmax
28,ID_001202,0.4,0.383545,134,360,212,434
29,ID_001202,0.5,0.472168,100,61,152,112
87,ID_001206,0.5,0.492432,118,58,185,126
178,ID_001213,0.3,0.298828,263,331,304,375
194,ID_001214,0.4,0.369141,370,252,418,290
...,...,...,...,...,...,...,...
7736,ID_001784,0.3,0.261719,105,89,123,105
7737,ID_001784,0.3,0.289795,231,377,272,440
7790,ID_001788,0.4,0.356689,55,393,124,460
7791,ID_001788,0.5,0.454590,63,54,125,119
